In [1]:
import sys
if '../working' not in sys.path: 
    sys.path.append('../working')

!sage -preparse ../working/normalizer.sage
!mv ../working/normalizer.sage.py ../working/normalizer.py

!sage -preparse ../working/self_similar.sage
!mv ../working/self_similar.sage.py ../working/self_similar.py

from normalizer import *
from self_similar import *
prepare_gap_env()

In [2]:

def permute(space, repeat) -> str:
    """ Get all the words of letters from the given
    space of given length.

    Parameters
    ----------
    space   : Iterable container of str elements of len 1.
    repeat  : int, length of yielded words
    allow_same_neighbours : if False then all words that don't have same elements
                            on the neighbour positions will be returned and all
                            possible words otherwise.
                            Default False
    Yields
    -------
    str, result word
    """
    if repeat == 1:
        for el in space:
            yield [el]
    elif repeat < 1:
        yield []
    else:
        for prev in permute(space, repeat - 1):
            for el in space:
                yield prev + [el]


def all_words(space, max_len=-1):
    """ Get all possible words (infinite set) of elements from the given
    space.

    Parameters
    ----------
    space   : Iterable container of str elements of len 1.
    allow_same_neighbours : if False then all words that don't have same elements
                            on the neighbour positions will be returned and all
                            possible words otherwise.
                            Default False.
    max_len : maximum allowed length of returned words.
              Default None, which means that there is no maximum length

    Yields
    -------
    str, result word
    """
    i = 1
    while True:
        for el in permute(space, repeat=i):
            yield el
        i += 1

        if max_len != -1 and i > max_len:
            break

def build_group(group_index, deep=3):
    e = matrix(QQ, [[1, 0], [0, 1]])
    G = to_L_basis(group_index, dim=2)
    gens = G.GeneratorsOfGroup()
    gens = [matrix(QQ, el) for el in gens]
    gens = [el for el in gens if el[:2, :2] != e]

    e = matrix(QQ, [[1, 0, 0], [0, 1, 0], [0, 0, 1]])

    alpha = {}
    snot = []
    for seq in all_words(gens, max_len=deep): 
        
        el = e 
        for el2 in seq: 
            el *= el2 
        
        g = el[:2, :2]
        t = el[:2, 2]
        if str(g) not in alpha:
            alpha[str(g)] = t
            snot.append(block_matrix([[g, t], [0, 1]]))

    P = G.PointGroup()
    P = [matrix(QQ, el) for el in P.AsList()]

    for mtx in P:
        assert str(mtx) in alpha, 'couldnt build every element of point group'

    return G, P, alpha, snot

In [3]:
group_index = 17

G, P, alpha, snot = build_group(group_index)
print('Point Group:')
print(ascii_art(P))

Point Group:
[ [1 0]  [-1  1]  [ 0 -1]  [-1  0]  [ 1 -1]  [0 1]  [ 1  0]  [-1  1]  [ 0 -1] 
[ [0 1], [-1  0], [-1  0], [ 0 -1], [ 1  0], [1 0], [ 1 -1], [ 0  1], [ 1 -1],

 [-1  0]  [ 1 -1]  [ 0  1] ]
 [-1  1], [ 0 -1], [-1  1] ]


In [4]:
print(ascii_art(snot))

[ [ 0 -1| 0]  [-1  0| 0]  [ 0 -1| 0]  [-1  1| 0]  [ 0  1| 0]  [ 1  0| 0] 
[ [ 1 -1| 0]  [ 0 -1| 0]  [-1  0| 0]  [-1  0| 0]  [-1  1| 0]  [ 1 -1| 0] 
[ [-----+--]  [-----+--]  [-----+--]  [-----+--]  [-----+--]  [-----+--] 
[ [ 0  0| 1], [ 0  0| 1], [ 0  0| 1], [ 0  0| 1], [ 0  0| 1], [ 0  0| 1],

 [1 0|0]  [0 1|0]  [-1  1| 0]  [ 1 -1| 0]  [-1  0| 0]  [ 1 -1| 0] ]
 [0 1|0]  [1 0|0]  [ 0  1| 0]  [ 1  0| 0]  [-1  1| 0]  [ 0 -1| 0] ]
 [---+-]  [---+-]  [-----+--]  [-----+--]  [-----+--]  [-----+--] ]
 [0 0|1], [0 0|1], [ 0  0| 1], [ 0  0| 1], [ 0  0| 1], [ 0  0| 1] ]


# Generate investigation for a group

In [11]:
group_index = 2

disp = 'latex'

if disp == 'latex':
    _display = latex
    title = '\\newpage\n\\subsection{Group %d}' % group_index
    section = '\\subsubsection{%s}'
    pref = '$$'
else: 
    _display = ascii_art
    title = f'=========================== Group {group_index} =================================='
    section = '\n----------------%s-------------------------\n'
    pref = '\n'

def display(*args, use_pref=True): 
    if use_pref:
        return pref + str(_display(*args)) + pref
    else: 
        return str(_display(*args))

print(title)
G, P, alpha, snot = build_group(group_index)
print('Generators of group:')
print(display([matrix(QQ, el) for el in G.GeneratorsOfGroup()]))
print('SNoT')
print(display(snot))

a0, a1 = var('a0 a1')
x = matrix([[a0], [a1]])
E = matrix(QQ, [[1, 0], [0, 1]])

print(section % 'Normalizers')   

norms = normalizers(group_index, to_l_basis=True, 
                    dim=2, verbose=False, ignore_trivial=True)

print(display(norms))

print(section % 'Dilation')   

if G.IsSymmorphicSpaceGroup(): 
    print(f"Group {group_index} is a semi-direct product, therefore the dilation part is trivial and only consists of integral vectors. ")


if disp == 'latex': 
    print('\\begin{enumerate}')
    pref2 = '\\item'
else: 
    pref2 = '...'


for A_inv in norms:
    print(pref2 + ' testing inverse A (should have integral entities):')
    print(display(A_inv))
    
    A = A_inv.inverse()
    conds = []
    variables = []
    coeffs = []
    if not G.IsSymmorphicSpaceGroup(): 
        for i, g in enumerate(P): 
            conj = (A_inv * g * A).simplify_rational()
            # conj = g
            condition = A_inv * alpha[str(g)] -  alpha[str(conj)]
            sym_res = (conj - E) * x
        
            # print(pref + '\\alpha(g) = ')
            # print(display(alpha[str(g)], use_pref=False) + pref)
            # print(pref + '\\alpha\\left(\\tau\\left(' + display(g) + '\\right)\\right) = ')
            
            alpha_conj = (E - conj) * x  + A_inv() * alpha[str(g)]
            # if disp == 'latex':
            #     print(pref + '(A^{-1}, a)' + display(block_matrix([[g, alpha[str(g)]], [0, 1]]), use_pref=False) + '(A, -Aa) = ')
            #     print(display(block_matrix([[conj, alpha_conj], [0, 1]]), use_pref=False) + '=')
            #     print(display(block_matrix([[conj, alpha[str(conj)]], [0, 1]]), use_pref=False) + pref)
            # else: 
            #     print(ascii_art('\na_inv\n\n') + ascii_art(' ')
            #           + ascii_art(block_matrix([[g, alpha[str(g)]], [0, 1]])) + ascii_art('\na\n\n') + ascii_art(' ')
            #           + ascii_art('\n=\n\n') + ascii_art(' ') + ascii_art(block_matrix([[conj, alpha_conj], [0,1]])) + ascii_art(' ')
            #           + ascii_art('\n=\n\n') + ascii_art(' ') 
            #           + ascii_art(block_matrix([[conj, alpha[str(conj)]], [0, 1]])))
            #     print()
    
            T = block_matrix([[A_inv, x], [0, 1]])
            if disp == 'latex':
                print(pref + '(A^{-1}, a)' + display(block_matrix([[g, alpha[str(g)]], [0, 1]]), use_pref=False) + '(A, -Aa) = ')
                print(display(block_matrix([[conj, alpha_conj], [0, 1]]), use_pref=False) + '=')
                print(display(block_matrix([[conj, alpha[str(conj)]], [0, 1]]), use_pref=False) + pref)
            else: 
                print(ascii_art('\na_inv\n\n') + ascii_art(' ')
                      + ascii_art(snot[i]) + ascii_art('\na\n\n') + ascii_art(' ')
                      + ascii_art('\n=\n\n') + ascii_art(' ') + ascii_art(T * snot[i] * T.inverse()) + ascii_art(' ')
                      + ascii_art('\n=\n\n') + ascii_art(' ') 
                      + ascii_art(block_matrix([[snot[i][:2, :2], alpha[str(snot[i][:2, :2])]], [0, 1]])))
                print()
                
            conds.append(sym_res[0][0] - (condition[0][0] + var(f'n{i}')))
            cond_coeffs = {}
            # for variable in conds[-1].arguments(): 
                # cur_coeff = conds[-1].coefficients(variable, sparse=False)
                # cond_coeffs[variable] = cur_coeff[1]
            # coeffs.append(cond_coeffs)
            
            conds.append(sym_res[1][0] - (condition[1][0] + var(f'm{i}')))
            # cond_coeffs = {}
            # for variable in conds[-1].arguments(): 
                # cur_coeff = conds[-1].coefficients(variable, sparse=False)
                # cond_coeffs[variable] = cur_coeff[1]
            # coeffs.append(cond_coeffs)
            
            variables.append(var(f'n{i}'))
            variables.append(var(f'm{i}'))
        
        tmp = [con == 0 for con in conds]
        print('\nequations: ')
        print(display(tmp))
        print('\nanswer:')
        res = solve(tmp, *variables)
        print(display(*res))
    
    print('Simplicity')
    print(pref + 'A = \n' + display(A.simplify_rational(), use_pref=False) + pref)
    print('\neigenvalues:')
    print(display([el[0] for el in A.charpoly().roots()]))
    print('charpoly:')
    chp = A.charpoly()(SR('x'))
    chp = factor(chp)
    print(display(chp))
    print('\nindex of subgroup:')
    print(pref + '[G : H] = \n' + display(A_inv.det(), use_pref=False) + pref)

if disp == 'latex': 
    print('\\end{enumerate}')

\newpage
\subsection{Group 2}
Generators of group:
$$\left[\left(\begin{array}{rrr}
-1 & 0 & 0 \\
0 & -1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & 1 \\
0 & 1 & 0 \\
0 & 0 & 1
\end{array}\right), \left(\begin{array}{rrr}
1 & 0 & 0 \\
0 & 1 & 1 \\
0 & 0 & 1
\end{array}\right)\right]$$
SNoT
$$\left[\left(\begin{array}{rr|r}
-1 & 0 & 0 \\
0 & -1 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right), \left(\begin{array}{rr|r}
1 & 0 & 0 \\
0 & 1 & 0 \\
\hline
 0 & 0 & 1
\end{array}\right)\right]$$
\subsubsection{Normalizers}
$$\left[\right]$$
\subsubsection{Dilation}
Group 2 is a semi-direct product, therefore the dilation part is trivial and only consists of integral vectors. 
\begin{enumerate}
\end{enumerate}


# Special ocasion: group 9

In [80]:
matrix(QQ, [[-1, -2], [2, 1]]).inverse()

[ 1/3  2/3]
[-2/3 -1/3]

In [12]:
A = matrix(QQ, [[0, 1/2], [1, 0]])
t = matrix(QQ, [[0], [0]])
t = block_matrix([[A, t], [0, 1]])
self_similar(1, t, dim=2, verbose=True, change_basis=True)


conjugate el:
[1 0 0]
[0 1 2]
[0 0 1]
conj in G: True

conjugate el:
[1 0 1]
[0 1 0]
[0 0 1]
conj in G: True
----------------------------------------------------
Index of subgroup H: 2
Transversal:
[1 0 0]
[0 1 0]
[0 0 1]

[1 0 0]
[0 1 1]
[0 0 1]
a_1d_1:
[1 0 1]
[0 1 0]
[0 0 1]

a_1d_2:
[1 0 1]
[0 1 1]
[0 0 1]

a_2d_1:
[1 0 0]
[0 1 1]
[0 0 1]

a_2d_2:
[1 0 0]
[0 1 2]
[0 0 1]



({('a_1', 1): (1, 'a_1a_2'),
  ('a_1', 2): (2, 'a_1a_2'),
  ('a_2', 1): (2, 'e'),
  ('a_2', 2): (1, 'e')},
 {'[1 0 1]\n[0 1 0]\n[0 0 1]': 'a_1',
  '[1 0 0]\n[0 1 1]\n[0 0 1]': 'a_2',
  '[ 1  0 -1]\n[ 0  1  0]\n[ 0  0  1]': 'a_1^(-1)',
  '[ 1  0  0]\n[ 0  1 -1]\n[ 0  0  1]': 'a_2^(-1)',
  '[1 0 0]\n[0 1 0]\n[0 0 1]': 'e'})